In [11]:
import pandas as pd
import matplotlib.pyplot as plt

## Fine-Tuning LLaVA on the RMET

We are looking at our results of fine-tuning the LLaVA model on the RMET task. We are looking at 5 different versions of the models: the base model and 4 fine-tuned models with different levels of training. 

### 1 RMET data

#### 1.1 Loading and checking the data

In [246]:
base1 = pd.read_csv('rmet_results/rmet_base-1.txt')
base2 = pd.read_csv('rmet_results/rmet_base-2.txt')
base3 = pd.read_csv('rmet_results/rmet_base-3.txt')
ep1_1 = pd.read_csv('rmet_results/rmet_1ep-1.txt')
ep1_2 = pd.read_csv('rmet_results/rmet_1ep-2.txt')
ep1_3 = pd.read_csv('rmet_results/rmet_1ep-3.txt')
ep1_4 = pd.read_csv('rmet_results/rmet_1ep-4.txt')
ep5_1 = pd.read_csv('rmet_results/rmet_5ep-1.txt')
ep5_2 = pd.read_csv('rmet_results/rmet_5ep-2.txt')
ep5_3 = pd.read_csv('rmet_results/rmet_5ep-3.txt')
ep7_1 = pd.read_csv('rmet_results/rmet_7ep-1.txt')
ep7_2 = pd.read_csv('rmet_results/rmet_7ep-2.txt')
ep7_3 = pd.read_csv('rmet_results/rmet_7ep-3.txt')
ep10_1 = pd.read_csv('rmet_results/rmet_10ep-1.txt')
ep10_2 = pd.read_csv('rmet_results/rmet_10ep-2.txt')
# ep10_3 = pd.read_csv('rmet_results/rmet_10ep-3.txt')

In [247]:
rmet = pd.concat([base1, base2, base3, ep1_1, ep1_2, ep1_3, ep1_4, ep5_1, ep5_2, ep5_3, ep7_1, ep7_2, ep7_3, ep10_1, ep10_2], axis=1)

In [248]:
rmet

,llava_base-1,llava_base-2,llava_base-3,llava_1ep-1,llava_1ep-2,llava_1ep-3,llava_1ep-4,llava_5ep-1,llava_5ep-2,llava_5ep-3,llava_7ep-1,llava_7ep-2,llava_7ep-3,llava_10ep-1,llava_10ep-2
0,bored,playful,bored,bored,bored,bored,bored,comforting,comforting,comforting,comforting,comforting,comforting,comforting,comforting
1,upset,upset,upset,upset,upset,upset,upset,upset,upset,upset,upset,upset,upset,upset,upset
2,convinced,convinced,convinced,convinced,convinced,convinced,convinced,convinced,convinced,convinced,convinced,convinced,convinced,convinced,convinced
3,insisting,insisting,insisting,insisting,insisting,insisting,insisting,insisting,insisting,insisting,insisting,insisting,insisting,insisting,insisting
4,worried,worried,worried,worried,worried,worried,worried,worried,worried,worried,worried,worried,worried,worried,worried
5,alarmed,aghast,alarmed,alarmed,alarmed,alarmed,alarmed,alarmed,alarmed,alarmed,alarmed,alarmed,alarmed,alarmed,alarmed
6,uneasy,uneasy,uneasy,uneasy,uneasy,uneasy,uneasy,uneasy,uneasy,uneasy,uneasy,uneasy,uneasy,uneasy,uneasy
7,despondent,despondent,despondent,despondent,despondent,despondent,despondent,despondent,despondent,despondent,despondent,despondent,despondent,despondent,despondent
8,horrified,preoccupied,preoccupied,preoccupied,preoccupied,preoccupied,preoccupied,horrified,horrified,horrified,preoccupied,preoccupied,preoccupied,preoccupied,preoccupied
9,bored,cautious,cautious,cautious,cautious,cautious,cautious,cautious,cautious,cautious,cautious,cautious,cautious,cautious,cautious


### 2 Task performance


#### 2.1 Load answers

In [238]:
answers_file = 'rmet_materials/answers.txt'
with open(answers_file, 'r') as file:
    answers = [line.strip() for line in file.readlines()]

In [239]:
answers[:10]

['playful',
 'upset',
 'desire',
 'insisting',
 'worried',
 'fantasizing',
 'uneasy',
 'despondent',
 'preoccupied',
 'cautious']

#### 2.2 Check responses

In [249]:
performance = rmet.copy()

In [250]:
for model in performance.columns:
    performance[model] = [1 if performance[model][i] == answers[i] else 0 for i in range(len(answers))]

#### 2.3 Calculate number correct

In [251]:
num_correct = performance.sum(axis=0)

In [252]:
num_correct

llava_base-1    17
llava_base-2    22
llava_base-3    20
llava_1ep-1     22
llava_1ep-2     20
llava_1ep-3     19
llava_1ep-4     17
llava_5ep-1     18
llava_5ep-2     23
llava_5ep-3     21
llava_7ep-1     22
llava_7ep-2     21
llava_7ep-3     22
llava_10ep-1    22
llava_10ep-2    21
dtype: int64

#### 2.4 Calculate Performance Increase (from base model)

In [221]:
increase_performance = pd.DataFrame(num_correct)
increase_performance.columns = ['num_correct']

In [222]:
# Merge model types
increase_performance['model'] = [idx.split('-')[0] for idx in increase_performance.index]

In [223]:
model_performance = increase_performance.groupby('model')['num_correct'].mean()

In [224]:
model_performance = model_performance.reset_index()
model_performance.columns = ['model', 'num_correct']

In [225]:
model_performance

,model,num_correct
0,llava_10ep,21.333333
1,llava_1ep,20.333333
2,llava_5ep,20.666667
3,llava_7ep,21.666667
4,llava_base,20.000000


In [226]:
model_performance['percent'] = model_performance['num_correct'].apply(lambda x: x / 36)
model_performance['improvement'] = model_performance['percent'].apply(lambda x: x - model_performance.iloc[4, 2])

In [227]:
model_performance

,model,num_correct,percent,improvement
0,llava_10ep,21.333333,0.592593,0.037037
1,llava_1ep,20.333333,0.564815,0.009259
2,llava_5ep,20.666667,0.574074,0.018519
3,llava_7ep,21.666667,0.601852,0.046296
4,llava_base,20.000000,0.555556,0.000000
